In [1]:
from modello import pgm_index
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.optimizers import Adam
import datetime

In [2]:
%load_ext tensorboard

In [3]:
with open('datipisa/segments_wiki_ts_1M_uint64.csv') as file:
    indice = pd.read_csv(file, header=2)
x_train = np.fromfile("datipisa/wiki_ts_1M_uint64", dtype=np.uint64)

x_train = x_train.reshape(len(x_train), 1)

In [4]:
index = [0]
for i in range(1, len(x_train)):
    if x_train[i] == x_train[i - 1]:
        index.append(index[i-1])
    else:
        index.append(i)

In [5]:
init = indice['key'].to_numpy().reshape(1, len(indice))
slope = indice['slope'].to_numpy().reshape(1, len(indice))
intercept = indice['intercept'].to_numpy().reshape(1, len(indice))
neuroni = len(indice)
pgm = pgm_index(neuroni, init, slope, intercept, True)

In [6]:
opt = Adam(lr=0.000001)

In [7]:
pgm.compile(loss='mean_absolute_error', optimizer=opt)

In [8]:
y_train = np.array(index).reshape(len(x_train), 1)

In [9]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, update_freq='batch')
pgm.fit(x_train, y_train, batch_size=50, callbacks=[tensorboard_callback], epochs=10, verbose=1)



Epoch 1/10
20000/20000 [==============================] - 62s 3ms/step - loss: 27.5712
Epoch 2/10
20000/20000 [==============================] - 62s 3ms/step - loss: 26.9811
Epoch 3/10
20000/20000 [==============================] - 61s 3ms/step - loss: 26.9026
Epoch 4/10
20000/20000 [==============================] - 62s 3ms/step - loss: 26.8760
Epoch 5/10
20000/20000 [==============================] - 62s 3ms/step - loss: 26.8614
Epoch 6/10
20000/20000 [==============================] - 64s 3ms/step - loss: 26.8537
Epoch 7/10
20000/20000 [==============================] - 64s 3ms/step - loss: 26.8460
Epoch 8/10
20000/20000 [==============================] - 62s 3ms/step - loss: 26.8421
Epoch 9/10
20000/20000 [==============================] - 61s 3ms/step - loss: 26.8387
Epoch 10/10
20000/20000 [==============================] - 65s 3ms/step - loss: 26.8368


In [10]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 4083), started 0:22:40 ago. (Use '!kill 4083' to kill it.)